# Lossless -> gridded 

Grids `last_trade` data: 

- Each `BinanceLastTradesGrid` reads the underlying lossless dataset `symbol_date_df` to determine symbol-date pairs. 
- Each dataset maintains a list of "validated" dates. Second runs will be very quick

In [ ]:
import mnemosyne as ms 
import polars as pl
from datetime import date as Date 

args = [
    (ms.DatasetType.BinanceSpotTrades, '4s', 'USDC'),
    (ms.DatasetType.BinanceSpotTrades, '10m', 'USDC'),

    (ms.DatasetType.BinanceUmPerpTrades, '4s', 'USDC'),
    (ms.DatasetType.BinanceUmPerpTrades, '10m', 'USDC'),

    (ms.DatasetType.BinanceSpotTrades, '20s', 'USDT'), 
    (ms.DatasetType.BinanceSpotTrades, '2m', 'USDT'),
    (ms.DatasetType.BinanceSpotTrades, '4s', 'USDT'),
    (ms.DatasetType.BinanceSpotTrades, '10m', 'USDT'),

    (ms.DatasetType.BinanceUmPerpTrades, '2m', 'USDT'),
    (ms.DatasetType.BinanceUmPerpTrades, '20s', 'USDT'),
    (ms.DatasetType.BinanceUmPerpTrades, '10m', 'USDT'),
    (ms.DatasetType.BinanceUmPerpTrades, '4s', 'USDT'),
]

In [7]:
for dataset_type, grid_interval, peg_symbol in args:
    # Reads the underlying lossless dataset's `symbol_date_df` to determine symbol-date pairs
    dataset = ms.binance.BinanceLastTradesGrid(
        peg_symbol=peg_symbol, 
        grid_interval=grid_interval, 
        dataset_type=dataset_type, 
        parquet_names='*.parquet', # Write to a single parquet: polars defaults to "0.parquet, 1.parquet ..."
        num_workers=2, 
    )
    print(f'{dataset_type} {grid_interval} {peg_symbol}: {dataset.path}')
    dataset.compute(recompute=False, days_per_batch=30)

BinanceSpotTrades 4s USDC: /data/mnemosyne/binance/grids/spot/last_trade/4s/peg_symbol=USDC
BinanceSpotTrades 10m USDC: /data/mnemosyne/binance/grids/spot/last_trade/10m/peg_symbol=USDC
BinanceUmPerpTrades 4s USDC: /data/mnemosyne/binance/grids/futures/um/last_trade/4s/peg_symbol=USDC
BinanceUmPerpTrades 10m USDC: /data/mnemosyne/binance/grids/futures/um/last_trade/10m/peg_symbol=USDC
BinanceSpotTrades 20s USDT: /data/mnemosyne/binance/grids/spot/last_trade/20s/peg_symbol=USDT
BinanceSpotTrades 2m USDT: /data/mnemosyne/binance/grids/spot/last_trade/2m/peg_symbol=USDT
BinanceSpotTrades 4s USDT: /data/mnemosyne/binance/grids/spot/last_trade/4s/peg_symbol=USDT
BinanceSpotTrades 10m USDT: /data/mnemosyne/binance/grids/spot/last_trade/10m/peg_symbol=USDT
BinanceUmPerpTrades 2m USDT: /data/mnemosyne/binance/grids/futures/um/last_trade/2m/peg_symbol=USDT
BinanceUmPerpTrades 20s USDT: /data/mnemosyne/binance/grids/futures/um/last_trade/20s/peg_symbol=USDT
BinanceUmPerpTrades 10m USDT: /data/mn

# Usage example

In [ ]:
peg_symbol = 'USDT'
dstype = ms.DatasetType.BinanceUmPerpTrades
grid_interval = '2m'

dataset = ms.binance.BinanceLastTradesGrid(
        peg_symbol=peg_symbol, 
        grid_interval=grid_interval, 
        dataset_type=dstype, 
        parquet_names='*.parquet', 
        num_workers=4, 
    )

# Reading the full dataset
lf = pl.scan_parquet(dataset.path / f'**/{dataset.parquet_names}')
dataset.universe_df

symbol,date
str,date
"""AGLD""",2024-03-17
"""APE""",2024-03-17
"""REN""",2024-03-17
"""JUP""",2024-03-17
"""AXS""",2024-03-17
…,…
"""DAM""",2025-10-23
"""TOKEN""",2025-10-23
"""VVV""",2025-10-23
